Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

## Summary:
### for two layer nn:
- reg_cof=0:       89.9%
- reg_cof=0.01:    90.2%
- reg_cof=0.001:   92.9%

code are shown below:

In [17]:
hidden_nodes_num = 1024
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes_num]))
  biases1 = tf.Variable(tf.zeros([hidden_nodes_num]))
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_nodes_num, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
    
  
  # Training computation.
  def forward_prop(data,layer):#define a forward propagation function to make the code more readable
    h1 = tf.nn.relu(tf.matmul(data, weights1) + biases1)
    if layer==1:   #output the first activation
        return h1
    elif layer==2:
        return tf.matmul(h1, weights2) + biases2
    else:
        print ('no layer'+str(layer)+' exists, please re-try another layer')
  
  logits = forward_prop(tf_train_dataset,2)             #logis is propate the train data 
  
  #======here is the changes, where the regularization is added =================
  reg = tf.reduce_mean(tf.nn.l2_loss(weights1)+tf.nn.l2_loss(weights2)+tf.nn.l2_loss(biases1)+tf.nn.l2_loss(biases2))
  reg_cof = 0.005
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))+reg_cof*reg
  #===========================================================================
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(forward_prop(tf_valid_dataset,2) )    #this is propagate the valid data
  test_prediction = tf.nn.softmax(forward_prop(tf_test_dataset,2))       #this is propagate the test data

In [18]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1906.973999
Minibatch accuracy: 8.6%
Validation accuracy: 28.0%
Minibatch loss at step 500: 128.100037
Minibatch accuracy: 83.6%
Validation accuracy: 81.1%
Minibatch loss at step 1000: 10.882737
Minibatch accuracy: 85.9%
Validation accuracy: 85.3%
Minibatch loss at step 1500: 1.343368
Minibatch accuracy: 89.8%
Validation accuracy: 85.2%
Minibatch loss at step 2000: 0.626616
Minibatch accuracy: 89.1%
Validation accuracy: 84.8%
Minibatch loss at step 2500: 0.582647
Minibatch accuracy: 85.2%
Validation accuracy: 84.5%
Minibatch loss at step 3000: 0.663500
Minibatch accuracy: 83.6%
Validation accuracy: 84.9%
Test accuracy: 91.5%


### for logistic model:
- reg_cof=0:       86.4%
- reg_cof=0.1:     84.7%
- reg_cof=0.01:    88.3%
- reg_cof=0.001:   88.7%

code are shown below:

In [27]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases

  #======here is the changes, where the regularization is added =================
  reg = tf.reduce_mean(tf.nn.l2_loss(weights)+tf.nn.l2_loss(biases))
  reg_cof = 0.001
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))+reg_cof*reg
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [28]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 22.215235
Minibatch accuracy: 7.8%
Validation accuracy: 9.9%
Minibatch loss at step 500: 2.445254
Minibatch accuracy: 81.2%
Validation accuracy: 76.2%
Minibatch loss at step 1000: 1.701244
Minibatch accuracy: 81.2%
Validation accuracy: 78.3%
Minibatch loss at step 1500: 0.980376
Minibatch accuracy: 82.0%
Validation accuracy: 80.0%
Minibatch loss at step 2000: 0.872170
Minibatch accuracy: 88.3%
Validation accuracy: 80.6%
Minibatch loss at step 2500: 0.766334
Minibatch accuracy: 78.9%
Validation accuracy: 81.2%
Minibatch loss at step 3000: 0.807648
Minibatch accuracy: 79.7%
Validation accuracy: 81.6%
Test accuracy: 88.7%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

### my answer:
in this problem, to achieve an extreme case of overfitting, I just make the NN training on the same small dataset (changes are shown in the second block);
final result is: the training accuracy is 100% but the validation accuracy and test accuracy is reletively small

In [29]:
hidden_nodes_num = 1024
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes_num]))
  biases1 = tf.Variable(tf.zeros([hidden_nodes_num]))
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_nodes_num, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
    
  
  # Training computation.
  def forward_prop(data,layer):#define a forward propagation function to make the code more readable
    h1 = tf.nn.relu(tf.matmul(data, weights1) + biases1)
    if layer==1:   #output the first activation
        return h1
    elif layer==2:
        return tf.matmul(h1, weights2) + biases2
    else:
        print ('no layer'+str(layer)+' exists, please re-try another layer')
  
  logits = forward_prop(tf_train_dataset,2)             #logis is propate the train data 
  
  reg = tf.reduce_mean(tf.nn.l2_loss(weights1)+tf.nn.l2_loss(weights2)+tf.nn.l2_loss(biases1)+tf.nn.l2_loss(biases2))
  reg_cof = 0.005
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))+reg_cof*reg
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(forward_prop(tf_valid_dataset,2) )    #this is propagate the valid data
  test_prediction = tf.nn.softmax(forward_prop(tf_test_dataset,2))       #this is propagate the test data

In [30]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    #offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    
    #====================here is the changes:====================
    offset = 2#any constant makes sense here. cause we will only train the model depends on the small data
    #============================================================
    
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1989.797363
Minibatch accuracy: 9.4%
Validation accuracy: 27.2%
Minibatch loss at step 500: 128.709122
Minibatch accuracy: 100.0%
Validation accuracy: 63.3%
Minibatch loss at step 1000: 10.550347
Minibatch accuracy: 100.0%
Validation accuracy: 66.5%
Minibatch loss at step 1500: 0.967026
Minibatch accuracy: 100.0%
Validation accuracy: 70.0%
Minibatch loss at step 2000: 0.185878
Minibatch accuracy: 100.0%
Validation accuracy: 69.9%
Minibatch loss at step 2500: 0.118824
Minibatch accuracy: 100.0%
Validation accuracy: 69.6%
Minibatch loss at step 3000: 0.111627
Minibatch accuracy: 100.0%
Validation accuracy: 69.5%
Test accuracy: 76.6%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

- in extrme overfitting case: improve by 0.4%, from 76.6% to 77%
- in general case (reg_cof=0.001): 


In [63]:
hidden_nodes_num = 1024
batch_size = 250

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes_num]))
  biases1 = tf.Variable(tf.zeros([hidden_nodes_num]))
  weights2 = tf.Variable(
    tf.truncated_normal([hidden_nodes_num, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  keep_prob = 0.5
  def forward_prop_drop(data,layer):#define a forward propagation function to make the code more readable
    h1 = tf.nn.relu(tf.matmul(data, weights1) + biases1)
    #============================here is the change!!======================
    #note that the dropout is only applied in hidden layer, not in the final layer!!
    #this is because dropout is meant to force the NN learn redundant info
    h1_drop = tf.nn.dropout(h1,keep_prob)
    if layer==1:   #output the first activation
        return h1_drop
    elif layer==2:
        return tf.matmul(h1_drop, weights2) + biases2
    else:
        print ('no layer'+str(layer)+' exists, please re-try another layer')
        
   #propagation computation for test and validate, not dropout performed!!
  def forward_prop(data,layer):#define a forward propagation function to make the code more readable
    h1 = tf.nn.relu(tf.matmul(data, weights1) + biases1)
    if layer==1:   #output the first activation
        return h1
    elif layer==2:
        return tf.matmul(h1, weights2) + biases2
    else:
        print ('no layer'+str(layer)+' exists, please re-try another layer')
  
  logits = forward_prop_drop(tf_train_dataset,2)             #logis is propate the train data 
  
  
  reg = tf.reduce_mean(tf.nn.l2_loss(weights1)+tf.nn.l2_loss(weights2)+tf.nn.l2_loss(biases1)+tf.nn.l2_loss(biases2))
  reg_cof = 0
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))+reg_cof*reg
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(forward_prop(tf_train_dataset,2))
  valid_prediction = tf.nn.softmax(forward_prop(tf_valid_dataset,2) )    #this is propagate the valid data
  test_prediction = tf.nn.softmax(forward_prop(tf_test_dataset,2))       #this is propagate the test data

TypeError: List of Tensors when single Tensor expected

In [62]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    
    #====================here is the changes:====================
    #offset = 2#any constant makes sense here. cause we will only train the model depends on the small data
    #============================================================
    
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 725.033813
Minibatch accuracy: 9.2%
Validation accuracy: 17.6%
Minibatch loss at step 500: 22.812178
Minibatch accuracy: 77.6%
Validation accuracy: 80.1%
Minibatch loss at step 1000: 14.310205
Minibatch accuracy: 74.8%
Validation accuracy: 80.5%
Minibatch loss at step 1500: 15.287352
Minibatch accuracy: 78.0%
Validation accuracy: 78.8%
Minibatch loss at step 2000: 8.991682
Minibatch accuracy: 81.6%
Validation accuracy: 79.4%
Minibatch loss at step 2500: 5.774158
Minibatch accuracy: 81.6%
Validation accuracy: 78.7%
Minibatch loss at step 3000: 6.187500
Minibatch accuracy: 78.8%
Validation accuracy: 79.0%
Test accuracy: 86.3%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
